#  Aula 11: Machine Learning (Cont...)

A base de dados 20 Newsgroups é uma coleção de, aproximadamente, 20 mil documentos com textos de notícias, distribuídas igualmente entre 20 tipos de tópicos.

Alguns dos tópicos são bastante similares como, por exemplo, comp.sys.ibm.pc.hardware e comp.sys.mac.hardware. Outros são bem destintos (misc.forsale e soc.religion.christian). Abaixo segue a lista completa dos 20 tópicos

``comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x	
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey	
sci.crypt
sci.electronics
sci.med
sci.space
misc.forsale	
talk.politics.misc
talk.politics.guns
talk.politics.mideast	
talk.religion.misc
alt.atheism
soc.religion.christian``

Como exemplo, podemos ver abaixo um desses textos:

### comp.sys.mac.hardware

From: narlochn@kirk.msoe.edu

Subject: General questions on software and hardware...

Distribution: usa

Organization: Milwaukee School Of Engineering, Milwaukee, WI USA

Lines: 18

I have just been introduced to Macintosh systems.
I have a few questions.  E-mail prefered.

1) Just what is System 7?  I want hard details not dingy commercials like their ads in magazines...

2) Has anyone used the Microsoft Office 3.0?  I would like suggestions on, and descriptions on:

* each has a microsoft before the actual name.

a) *Word 5.1:

b) *Excel 4.0:

c) *Power Point:

d) *Mail 3.1

3)  What is the major differences between Mac Wordperfect and Word?

Thanks a lot!


### rec.sport.baseball

From: zappala@pollux.usc.edu (Daniel Zappala)

Subject: Angels win!

Article-I.D.: pollux.1psvouINNa2l

Distribution: world

Organization: University of Southern California, Los Angeles, CA

Lines: 6

NNTP-Posting-Host: pollux.usc.edu


The Angels won their home opener against the Brewers today before 33,000+ 
at Anaheim Stadium, 3-1 on a 3-hitter by Mark Langston.  J.T. Snow and 
Gary Discarcina hit home runs for the Angels.

Daniel

### Objetivo:

Montar um modelo de Machine learning para prever em qual dos 20 grupos um novo texto pertence. 

In [1]:
import json
import logging
import pickle
import shutil
import re
import codecs
import csv
import os, errno
import tarfile
import time
import sys
import numpy as np
import pandas as pd
import scipy.sparse as sp
from pprint import pprint
from random import shuffle
import unicodedata
import scipy.sparse as sp

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import joblib

In [3]:
from itertools import islice

Podemos ler um arquivo PKZ para carregar os dados dos textos na memória. .PKZ é uma versão compactada de um arquivo pickle .PKL. Equivale a um arquivo .ZIP

In [4]:
with open("/Users/gustavorsampaio/Documents/Cursos/Python 2021.2/Aulas/20news-bydate.pkz", 'rb') as f:
    compressed_content = f.read()

In [5]:
uncompressed_content = codecs.decode(compressed_content, 'zlib_codec')
cache = pickle.loads(uncompressed_content)

ModuleNotFoundError: No module named 'sklearn.datasets.base'

In [5]:
len(cache)

2

In [ ]:
data_train = cache['train']
data_test = cache['test']

In [9]:
print(list(data_train))
print("\n")
print(list(data_test))

['target', 'DESCR', 'target_names', 'data', 'filenames']


['target', 'DESCR', 'target_names', 'data', 'filenames']


In [66]:
data_train["target"]

array([ 9,  4, 11, ..., 16, 18,  4])

In [62]:
data_train["target_names"]

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [64]:
data_train["data"]

["From: cubbie@garnet.berkeley.edu (                               )Subject: Re: Cubs behind Marlins? How?Article-I.D.: agate.1pt592$f9aOrganization: University of California, BerkeleyLines: 12NNTP-Posting-Host: garnet.berkeley.edugajarsky@pilot.njin.net writes:morgan and guzman will have era's 1 run higher than last year, and the cubs will be idiots and not pitch harkey as much as hibbard. castillo won't be good (i think he's a stud pitcher)       This season so far, Morgan and Guzman helped to lead the Cubs       at top in ERA, even better than THE rotation at Atlanta.       Cubs ERA at 0.056 while Braves at 0.059. We know it is early       in the season, we Cubs fans have learned how to enjoy the       short triumph while it is still there.",
 'From: gnelson@pion.rutgers.edu (Gregory Nelson)Subject: Thanks Apple: Free Ethernet on my C610!Article-I.D.: pion.Apr.6.12.05.34.1993.11732Organization: Rutgers Univ., New Brunswick, N.J.Lines: 26Well, I just got my Centris 610 yesterday.  It

In [15]:
data_train["filenames"]

array(['/Users/gustavorsampaio/Documents/Machine & Deep Learning/Machine Learning/Text/20news-bydate/20news-bydate-train/rec.sport.baseball/102736',
       '/Users/gustavorsampaio/Documents/Machine & Deep Learning/Machine Learning/Text/20news-bydate/20news-bydate-train/comp.sys.mac.hardware/50485',
       '/Users/gustavorsampaio/Documents/Machine & Deep Learning/Machine Learning/Text/20news-bydate/20news-bydate-train/sci.crypt/15246',
       ...,
       '/Users/gustavorsampaio/Documents/Machine & Deep Learning/Machine Learning/Text/20news-bydate/20news-bydate-train/talk.politics.guns/54663',
       '/Users/gustavorsampaio/Documents/Machine & Deep Learning/Machine Learning/Text/20news-bydate/20news-bydate-train/talk.politics.misc/178534',
       '/Users/gustavorsampaio/Documents/Machine & Deep Learning/Machine Learning/Text/20news-bydate/20news-bydate-train/comp.sys.mac.hardware/51858'],
      dtype='<U143')

Podemos ler os arquivos a partir de duas pastas. Uma contendo arquivos para treinamento e outra contendo os arquivos para a validação do nosso modelo.

In [6]:
data_home = "/Users/gustavorsampaio/Documents/Cursos/Python 2018.2/Aulas/20news-bydate/"
TRAIN_FOLDER = "/Users/gustavorsampaio/Documents/Cursos/Python 2018.2/Aulas/20news-bydate/20news-bydate-train"
TEST_FOLDER = "/Users/gustavorsampaio/Documents/Cursos/Python 2018.2/Aulas/20news-bydate/20news-bydate-test"

cache = dict(train=load_files(TRAIN_FOLDER, encoding='latin1'), test=load_files(TEST_FOLDER, encoding='latin1'))

In [15]:
list(cache)

['train', 'test']

In [20]:
cache['train']["target"]

array([ 9,  4, 11, ..., 16, 18,  4])

In [ ]:
compressed_content = codecs.encode(pickle.dumps(cache), 'zlib_codec')
uncompressed_content = codecs.decode(compressed_content, 'zlib_codec')
cache = pickle.loads(uncompressed_content)

In [19]:
list(cache)

['train', 'test']

In [7]:
data_train = cache['train']
data_test = cache['test']

In [8]:
print(list(data_train))
print("\n")
print(list(data_test))

['data', 'filenames', 'target_names', 'target', 'DESCR']


['data', 'filenames', 'target_names', 'target', 'DESCR']


In [9]:
data_train["data"][10]

'From: maler@vercors.imag.fr (Oded Maler)\nSubject: Re: FLAME and a Jewish home in Palestine\nNntp-Posting-Host: pelvoux\nOrganization: IMAG, University of Grenoble, France\nLines: 40\n\nIn article <C5HJBC.1HC@bony1.bony.com>, jake@bony1.bony.com (Jake Livni) writes:\n|> In article <1993Apr13.172422.2407@newshub.ariel.yorku.ca> nabil@ariel.yorku.ca (Nabil Gangi) writes:\n|> \n|> >According to Exodus, there were 600,000 Jews that marched out of Egypt.\n|> \n|> This is only the number of adult males.  The total number of Jewish\n|> slaves leaving Egypt was much larger.\n|> \n|> >The number which could have arrived to the Holy Lands must have been\n|> >substantially less ude to the harsh desert and the killings between the\n|> >Jewish tribes on the way..\n|> >\n|> >NABIL\n|> \n|> Typical Arabic thinking.  If we are guilty of something, so is\n|> everyone else.  Unfortunately for you, Nabil, Jewish tribes are not\n|> nearly as susceptible to the fratricidal murdering that is still so\n|> c

### Limpeza da base

Devemos realizar uma limpeza nos textos para eliminar quebra de linhas, tabs, ...

Tanto na base de treinamento:

In [10]:
data_train.data = [w.replace("\r\n", " ") for w in data_train.data]
data_train.data = [w.replace("\t", "") for w in data_train.data]
data_train.data = [w.replace("\n", "") for w in data_train.data]
data_train.data = [w.replace(">", "") for w in data_train.data]

In [11]:
data_train["data"][10]

'From: maler@vercors.imag.fr (Oded Maler)Subject: Re: FLAME and a Jewish home in PalestineNntp-Posting-Host: pelvouxOrganization: IMAG, University of Grenoble, FranceLines: 40In article <C5HJBC.1HC@bony1.bony.com, jake@bony1.bony.com (Jake Livni) writes:| In article <1993Apr13.172422.2407@newshub.ariel.yorku.ca nabil@ariel.yorku.ca (Nabil Gangi) writes:| | According to Exodus, there were 600,000 Jews that marched out of Egypt.| | This is only the number of adult males.  The total number of Jewish| slaves leaving Egypt was much larger.| | The number which could have arrived to the Holy Lands must have been| substantially less ude to the harsh desert and the killings between the| Jewish tribes on the way..| | NABIL| | Typical Arabic thinking.  If we are guilty of something, so is| everyone else.  Unfortunately for you, Nabil, Jewish tribes are not| nearly as susceptible to the fratricidal murdering that is still so| common among Arabs in the Middle East.  There were no " killings| betwee

quanto na base de validação:

In [12]:
data_test.data = [w.replace("\r\n", " ") for w in data_test.data]
data_test.data = [w.replace("\t", "") for w in data_test.data]
data_test.data = [w.replace("\n", "") for w in data_test.data]

### Descrever os dados:

In [14]:
print('Existem nesta base de treinamento ' + str(len(data_train.target_names)) + ' classes:\n')
print("Elas São ... \n")
print('\t', data_train.target_names, '\n')

Existem nesta base de treinamento 20 classes:

Elas São ... 

	 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc'] 



In [28]:
data_train.data[100]

'From: sera@zuma.UUCP (Serdar Argic)Subject: Role of \'SDPA.ORG\' in slaughter of Gunduz, Ariyak, Arikan, Benler,...Reply-To: sera@zuma.UUCP (Serdar Argic)Distribution: worldLines: 107In article <1993Apr16.044001.15540@urartu.sdpa.org hla@urartu.sdpa.org  writes:Sure it is. It tells us how far right the whole Turkish political spectrum Nobody ever exposed your crimes like that before? What was your personal role in the murder of Orhan Gunduz and Kemal Arikan, again? How many moreMuslims will be slaughtered by \'SDPA.ORG\' as publicly declared and filedwith legal authorities? Please spell it out for us. "...that more people have to die..."                     SDPA <91@urartu.UUCP  "Yes, I stated this and stand by it."                    SDPA <255@urartu.UUCP    January 28, 1982 - Los AngelesKemal Arikan is slaughtered by two Armenians while driving to work.     March 22, 1982 - Cambridge, MassachusettsPrelude to grisly murder. A gift and import shop belonging toOrhan Gunduz is blown up.

### Processando os textos para estimação

Extrair palavras do texto e converter para vetores numéricos:

max_df: Eliminar palavras nos documentos que aparecam com uma frequência muito elevada. Elas não irão adicionar muito ao nosso modelo.

In [20]:
count_vect = CountVectorizer(max_df=0.75) 
X_train_counts = count_vect.fit_transform(data_train.data)

In [21]:
X_train_counts

<11314x208687 sparse matrix of type '<class 'numpy.int64'>'
	with 1727239 stored elements in Compressed Sparse Row format>

In [23]:
list(X_train_counts)

[<1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 77 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 163 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 976 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 172 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 81 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 51 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 66 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	with 212 stored elements in Compressed Sparse Row format>,
 <1x208687 sparse matrix of type '<class 'numpy.int64'>'
 	w

In [24]:
X_train_counts.shape

(11314, 208687)

In [25]:
print('Nos ' + str(X_train_counts.shape[0]) + ' processos de treinamento temos ' + str(X_train_counts.shape[1]) + ' palavras')

Nos 11314 processos de treinamento temos 208687 palavras


Listar Palavras que mais aparecem

In [26]:
list(islice(count_vect.vocabulary_.items(), 20))

[('cubbie', 69155),
 ('garnet', 91515),
 ('berkeley', 51390),
 ('edu', 78709),
 ('re', 155722),
 ('cubs', 69167),
 ('behind', 50729),
 ('marlins', 123975),
 ('how', 101163),
 ('article', 45355),
 ('agate', 38135),
 ('1pt592', 10663),
 ('f9aorganization', 84608),
 ('university', 191572),
 ('california', 58169),
 ('berkeleylines', 51395),
 ('12nntp', 5255),
 ('posting', 149056),
 ('host', 101001),
 ('edugajarsky', 78950)]

Imaginem que textos mais longos possuem mais palavras e, desta maneira, teriam maior peso.

Para eliminar tal problema, iremos, ao invés de utilizar número de palavras, utilizar a sua frequência dentro de um texto

TfidfTransformer: Transformas uma matriz de contagem em uma matriz de frequências.

``Transform a count matrix to a normalized tf or tf-idf representation. Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency``

In [29]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 208687)

Listar Palavras que mais aparecem com maior detalhes

In [36]:
count_vect = CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 3))
X_train_counts = count_vect.fit_transform(data_train.data)
list(islice(count_vect.vocabulary_.items(), 20))

[('cubbie', 747970),
 ('garnet', 1092893),
 ('berkeley', 467883),
 ('edu', 903623),
 ('cubs', 748069),
 ('marlins', 1560955),
 ('article', 380623),
 ('agate', 301738),
 ('1pt592', 86506),
 ('f9aorganization', 992359),
 ('university', 2519722),
 ('california', 551346),
 ('berkeleylines', 468214),
 ('12nntp', 38420),
 ('posting', 1894111),
 ('host', 1232244),
 ('edugajarsky', 915364),
 ('pilot', 1853288),
 ('njin', 1703228),
 ('net', 1684652)]

In [37]:
list(islice(count_vect.vocabulary_.items(), 20))

[('cubbie', 747970),
 ('garnet', 1092893),
 ('berkeley', 467883),
 ('edu', 903623),
 ('cubs', 748069),
 ('marlins', 1560955),
 ('article', 380623),
 ('agate', 301738),
 ('1pt592', 86506),
 ('f9aorganization', 992359),
 ('university', 2519722),
 ('california', 551346),
 ('berkeleylines', 468214),
 ('12nntp', 38420),
 ('posting', 1894111),
 ('host', 1232244),
 ('edugajarsky', 915364),
 ('pilot', 1853288),
 ('njin', 1703228),
 ('net', 1684652)]

In [38]:
count_vect1 = CountVectorizer(ngram_range=(1, 3))
X_train_counts1 = count_vect1.fit_transform(data_train.data)
list(islice(count_vect1.vocabulary_.items(), 20))

[('from', 1215625),
 ('cubbie', 880046),
 ('garnet', 1254600),
 ('berkeley', 598387),
 ('edu', 1024110),
 ('subject', 2672065),
 ('re', 2369551),
 ('cubs', 880145),
 ('behind', 588450),
 ('marlins', 1810643),
 ('how', 1428578),
 ('article', 473633),
 ('agate', 305573),
 ('1pt592', 84968),
 ('f9aorganization', 1114970),
 ('university', 3088863),
 ('of', 2027637),
 ('california', 697664),
 ('berkeleylines', 598716),
 ('12nntp', 37822)]

Podemos construir um DataFrame com Palavras que mais aparecem

Função ravel():

``x = np.array([[1, 2, 3], [4, 5, 6]])

np.ravel(x)

array([1, 2, 3, 4, 5, 6])``

In [40]:
x = np.array([[1, 2, 3], [4, 5, 6]])

In [41]:
x

array([[1, 2, 3],
       [4, 5, 6]])

In [42]:
np.ravel(x)

array([1, 2, 3, 4, 5, 6])

In [43]:
occ = np.asarray(X_train_counts.sum(axis=0)).ravel().tolist()
occ

[1170,
 154,
 44,
 10,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 8,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 17,
 16,
 9,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 19,
 10,
 9,
 1,
 1,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 26,
 6,
 9,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 5,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 6,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 4,
 1,
 3,
 1,
 1,
 11,
 1,
 10,
 20,
 2,
 3,
 1,
 1,
 3,
 1,
 2,
 3,
 2,
 1,
 1,
 1,
 1,
 4,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 12,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 2,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,

In [44]:
counts_df = pd.DataFrame({'term': count_vect.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

,term,occurrences
424791,ax,62348
425827,ax ax,56278
426117,ax ax ax,50640
903623,edu,16825
651791,com,10215
2681047,writes,7833
380623,article,6971
862211,don,5787
1826092,people,5524
1468601,like,5496


Listar Palavras que mais aparecem com maior detalhes por frequência pesando tamanho do texto

In [47]:
tfidf_transformer = TfidfTransformer(use_idf=False) # Enable inverse-document-frequency reweighting
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 2739024)

Dataframe com Palavras que mais aparecem

In [48]:
weights = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': count_vect.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
903623,edu,0.066476
651791,com,0.039693
2681047,writes,0.030177
380623,article,0.026200
1894111,posting,0.023641
1232244,host,0.022182
1894482,posting host,0.021689
2519722,university,0.019357
862211,don,0.018487
1468601,like,0.018354


### Construir primeiro modelo de ML

In [49]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 3))),
                    ('tfidf', TfidfTransformer(use_idf=False)),
                    ('clf', MultinomialNB(fit_prior=False)), #Whether to learn class prior probabilities or not. If false, a uniform prior will be used.
                    ])

In [50]:
text_clf

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.75, ngram_range=(1, 3),
                                 stop_words='english')),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(fit_prior=False))])

In [51]:
text_clf_ok = text_clf.fit(data_train.data, data_train.target)

In [52]:
text_clf_ok

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.75, ngram_range=(1, 3),
                                 stop_words='english')),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(fit_prior=False))])

In [53]:
predictedNB = text_clf_ok.predict(data_test.data)

In [55]:
predictedNB

array([10, 16, 10, ..., 13,  3,  0])

In [56]:
data_test.target

array([10, 16, 14, ...,  4,  6,  7])

In [58]:
np.mean(predictedNB == data_test.target)

0.7842538502389803

In [65]:
data_test.data[5001]

"From: jbrown@vax.cns.muskingum.eduSubject: ESPN cares less about hockeyOrganization: Muskingum CollegeLines: 23ESPN is pathetic, I have been watching everybody complain about ESPN's coverageand I agree with all of you.  Might I suggest that we are getting all worked upover nothing.  We ALL knew that ESPN's coverage was going to suck.  I mean haveyou ever watched during the regular season Sportscenter coverage of hockey? Itsucks, I mean really sucks!!!  ESPN does not know hockey if it shot them in theass.  I was really pissed of about last night, coverage.  I go to school in OhioI *know* the Indians suck, so why show them?  If I were Bill Clement I wouldshoot myself.Anyway after I calmed down after I realized that they were not going to showovertime of the Caps Isles game I thought to myself I knew that this was goingto happen, in fact I remerbed myself praying that at least they would keep eventhe boring Devils game.Overall I think ESPN does not care if they show hockey or not and I f

In [66]:
andre = "From: jbrown@vax.cns.muskingum.eduSubject: ESPN cares less about hockeyOrganization: Muskingum CollegeLines: 23ESPN is pathetic, I have been watching everybody complain about ESPN's coverageand I agree with all of you.  Might I suggest that we are getting all worked upover nothing.  We ALL knew that ESPN's coverage was going to suck.  I mean haveyou ever watched during the regular season Sportscenter coverage of hockey? Itsucks, I mean really sucks!!!  ESPN does not know hockey if it shot them in theass.  I was really pissed of about last night, coverage.  I go to school in OhioI *know* the Indians suck, so why show them?  If I were Bill Clement I wouldshoot myself.Anyway after I calmed down after I realized that they were not going to showovertime of the Caps Isles game I thought to myself I knew that this was goingto happen, in fact I remerbed myself praying that at least they would keep eventhe boring Devils game.Overall I think ESPN does not care if they show hockey or not and I firmlybelive that they have know respect for the fans. ie *because of contract       obligations*, screw you ESPN.Praise be to all hockey fans who put up with this shit!GO HAWKS!!!JB"

In [67]:
text_clf_ok.predict([andre])

array([10])

In [68]:
data_test.target_names[10]

'rec.sport.hockey'

In [69]:
data_test.data[5000]

'From: jmeritt@mental.mitre.orgSubject: Rawlins has been listening to the DevilOrganization: UTexas Mail-to-News GatewayLines: 5NNTP-Posting-Host: cs.utexas.eduGod ItSelf appeared to me and spoke to me, saying "Rawlins has been listening toa deamon, and has been taken in by its satanic words!"Now, how we tell which divine inspiration comes from the One True God and whichcomes from a satanic trickster?'

In [70]:
data_test.target[5000]

19

In [72]:
data_test.target_names[19]

'talk.religion.misc'

In [75]:
#texto = 'From: stimpy@dev-null.phys.psu.edu (Gregory Nagy)Subject: Re: ESPN UP YOURS .........Organization: Penn State Laboratory for Elementary Steam PhysicsLines: 52NNTP-Posting-Host: dev-null.phys.psu.eduIn article <C5u542.3CD@news.udel.edu> tmavor@earthview.cms.udel.edu writes:>>>>[Various justifiable rantings on ESPN coverage by several deleted]>>>>The only way to change ESPN\'s thinking, if it is even possible, is to complain>to them directly.  Anyone know there telephone # in Bristol, Ct?  Heh... Try the rec.autos.sport FAQ. They are always calling ESPN to complain.I\'m sure you could find the number for ABC there too, as many west-coast viewers were compaining about how something as boring as hockey cut intothe Long Beach GP. =)>>I do find it hard to believe that ESPN doesn\'t think viewers will simply>change the channel from a boring game....I know I did.  And then, when>they didn\'t show the NYI-Wash overtime(s), I was livid!  If I wanted>to watch baseball, I could have turned on the Phillies-Padres extra>inning game....instead, I went to bed angry......I boycotted ESPN\'s>morning Sportscenter today, I was still so incensed.Were you (and several of the other people here it seems) asleep the day"contracts" were explained? ASPN has a piece of paper saying it MUSTshow that baseball game if it happens. Many businesses payedd money tohave their commercials run during a baseball game. This is a business,not your own personal video servant.>>My wife says I shouldn\'t go to bed angry, but last nite.........GRRRRRRR!>Maybe you should put that anger into something positive. For example, I sawads for the new Dodge both on the ESPN and KBL broadcasts. Why not write toDodge saying that "thanks to the ads run during the STANLEY CUP PLAYOFFS, you will now concider their products in the future. They love to hear stufflike that and in the future will be more willing to buy commercial timefor hockey games, giving ESPN (and other networks) more incentive to carrygames (just one example)Come on people, as great as we think it is, Hockey does not leapfrog the"big three" overight.> >--------------------------------------------------------------------->Tim Mavor   |  "I am known by many names.......>College of Marine Studies  |   some call me.........Tim.">Univ. of Delaware   |    >Newark, DE 19716   |  "You know much that is hidden, O\' Tim!">tmavor@pandora.cms.udel.edu|  Monty Python and the Holy Grail--------------------------------------------------------------------->'
#texto1 = 'One very good player played in the national baseball team'
texto = 'From: JEK@cu.nih.gov Subject: Mary, "blessed among women" Lines: 23  Dave Bernard writes:   > When Elizabeth greeted Mary with the words: "Blessed art thou  > among women" (Luke 1:42), it appears that this places Mary  > beyond the sanctification of normal humanity.  But Deborah says (Judges 5:24):   > Blessed among women shall be Jael the wife of Heber the Kenite,  > Blessed above all women in the tents.  It can doubtless be taken that Jael\'s slaying of Sisera was a type of Mary\'s victory over sin. But even if we take Deborah\'s words as applying prophetically or symbolically to Mary, they must still be applicable literally to Jael.  We may well take them to mean that God used her as a part of His plan for the deliverance of His people, and that she has this in common with Mary.  But we have no reason to suppose that they mean that she was sinless, and thus no reason to take the like expression applied to Mary as proof that she was sinless.   Yours,  James Kiefer'

In [84]:
texto1 = 'Python is a very cool language. We can use it to program computers to do that we want. We can also break the computer motherboard if we perform very fast actions. Also, we can collect sports information from the web. If we collect the data with a Machintoch computer things will be much faster. Macs are the best computers in the market. Mac computers do not break. Apple is a very strong company'

In [85]:
predictedNB = text_clf_ok.predict([texto1])

In [86]:
predictedNB

array([4])

In [87]:
data_test.target_names[4]

'comp.sys.mac.hardware'

In [80]:
data_test.target[2]

14

In [89]:
data_test.target_names[11]

'sci.crypt'

In [88]:
text_clf_ok.predict_proba([texto1])

array([[0.02046246, 0.07715548, 0.05456115, 0.09422573, 0.16198716,
        0.04684915, 0.05151794, 0.04158451, 0.02538419, 0.01615671,
        0.01748634, 0.12140565, 0.07492395, 0.03675703, 0.04011326,
        0.02528253, 0.03602724, 0.01844511, 0.02407457, 0.01559983]])

### Construir segundo modelo de ML 

In [94]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultinomialNB(fit_prior=False)),
])

parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'vect__max_df': (0.75, 0.80),
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-5), # Additive (Laplace/Lidstone) smoothing parameter
}

In [95]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1) # Number of jobs to run in parallel. None means 1. -1 means using all processors

In [96]:
gs_clf

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(stop_words='english')),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultinomialNB(fit_prior=False))]),
             n_jobs=-1,
             param_grid={'clf__alpha': (0.01, 1e-05),
                         'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.75, 0.8),
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)]})

In [97]:
gs_clf_ok = gs_clf.fit(data_train.data, data_train.target)

In [98]:
gs_clf.best_score_

0.9088743180671808

In [100]:
gs_clf.best_params_

{'clf__alpha': 0.01,
 'tfidf__use_idf': False,
 'vect__max_df': 0.75,
 'vect__ngram_range': (1, 1)}

In [ ]:
text_clf_final = Pipeline([('vect', CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 1))),
                    ('tfidf', TfidfTransformer(use_idf=False)),
                    ('clf', MultinomialNB(fit_prior=False, alpha=0.01)), #Whether to learn class prior probabilities or not. If false, a uniform prior will be used.
                    ])

In [101]:
predicted = gs_clf.predict(data_test.data)
print('Taxa de acerto: ', np.mean(predicted == data_test.target))

Taxa de acerto:  0.8347052575677111


### Construir terceiro modelo de ML 

In [103]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=15, random_state=42)), # loss: The loss function to be used, penalty: The penalty (aka regularization term) to be used, n_iter: The number of passes over the training data, random_state: The seed of the pseudo random number generator to use when shuffling the data
])
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
                  'vect__max_df': (0.75, 0.80),
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-5),
}

In [104]:
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

In [105]:
gs_clf_svm_ok = gs_clf_svm.fit(data_train.data, data_train.target)

In [106]:
gs_clf_svm_ok.best_score_

0.9238997551429934

In [107]:
gs_clf_svm_ok.best_params_

{'clf-svm__alpha': 1e-05,
 'tfidf__use_idf': True,
 'vect__max_df': 0.75,
 'vect__ngram_range': (1, 2)}

In [109]:
predicted

array([10, 16,  3, ...,  3,  3,  7])

In [110]:
data_test.target

array([10, 16, 14, ...,  4,  6,  7])

In [111]:
predicted == data_test.target

array([ True,  True, False, ..., False, False,  True])

In [112]:
np.mean([10,9,10])

9.666666666666666

In [113]:
from sklearn.metrics import accuracy_score

In [114]:
print(accuracy_score(data_test.target, predicted))

0.8487785448751991


In [115]:
gs_clf_svm_ok.predict([texto1])

array([4])

In [116]:
predicted = gs_clf_svm_ok.predict(data_test.data)
print('Taxa de acerto: ', np.mean(predicted == data_test.target))

Taxa de acerto:  0.8487785448751991


Montar modelo com melhores parâmetros

In [98]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 3))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf-svm', SGDClassifier(loss='squared_hinge', penalty='elasticnet', alpha=1e-5, max_iter=90, random_state=42)),
])
text_clf_svm = text_clf_svm.fit(data_train.data, data_train.target)
predicted_svm = text_clf_svm.predict(data_test.data)
np.mean(predicted_svm == data_test.target)

/Users/gustavorsampaio/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.5877588953797133

In [118]:
data_test.target

array([10, 16, 14, ...,  4,  6,  7])

In [119]:
le = preprocessing.LabelEncoder()
le.fit(data_train.target_names)

LabelEncoder()

In [120]:
joblib.dump(le, '/Users/gustavorsampaio/Documents/Cursos/Python 2020.2/Aulas/predictor_label.pkl')
joblib.dump(gs_clf_svm_ok, '/Users/gustavorsampaio/Documents/Cursos/Python 2020.2/Aulas/predictor_model.pkl')

['/Users/gustavorsampaio/Documents/Cursos/Python 2020.2/Aulas/predictor_model.pkl']

# Prever dados Não Classificados:

In [111]:
from sklearn.preprocessing import LabelEncoder
import joblib

grs_clf = joblib.load('/Users/gustavorsampaio/Documents/Cursos/Python 2019.2/aulas/predictor_model.pkl')
grs_le = joblib.load('/Users/gustavorsampaio/Documents/Cursos/Python 2019.2/aulas/predictor_label.pkl')

### Colocar os textos numa lista para passar ao classificador.

In [112]:
#p = data_test.data[0:10]
p = ['My computer runs faster than yours. It has 8 gb of RAM and it is a Macbook Pro. Macs ate the best']

In [107]:
p

['My computer runs faster than yours. It has 8 gb of RAM and it is a Macbook Pro. Macs ate the best']

In [114]:
predicted_model = grs_clf.predict(p)

In [115]:
predicted_model

array([4])

In [116]:
predicted_labels = grs_le.inverse_transform(predicted_model)

In [119]:
print(predicted_labels[0])

comp.sys.mac.hardware
